# Apply Elo Ratings
Using the research gathered in the "elo.ipynb" we will now apply the Elo ratings to the prepped dataset

Import required libraries

In [14]:
import pandas as pd

Create required functions

In [15]:
def calculate_expected_probability(team1_current_elo, team2_current_elo):
    r1 = team1_current_elo
    r2 = team2_current_elo
    return 1 / (1 + pow(10, ((r2 - r1) / 400)))

def calculate_rating_change(team1_current_elo, team2_current_elo, result):
    e1 = calculate_expected_probability(team1_current_elo, team2_current_elo)
    r1 = team1_current_elo
    k = 16
    s1 = result
    return r1 + k * (s1 - e1)

def get_winner(match):
    """`match` is a row in the dataset"""
    if match.home_final_score > match.away_final_score:
        return match.home_team
    elif match.home_final_score < match.away_final_score:
        return match.away_team
    else:
        return "Draw" 
    
def get_result_value(team_name, winner):
    if team_name == winner:
        return 1
    elif winner == "Draw":
        return 0.5
    else:
        return 0

Import data

In [16]:
matches = pd.read_pickle("../data/aflmatches_wo_elo.pypickle")

Set initial values

In [17]:
# Initialise Columns
matches['home_team_current_rating'] = 1500
matches['away_team_current_rating'] = 1500
matches['home_team_new_rating'] = 1500
matches['away_team_new_rating'] = 1500

# Create dict to hold current ratings
current_ratings = {team: 1500 for team in matches.home_team.unique()}

Calculate Elo Rating

In [18]:
# Calculate winners
matches['winner'] = matches.apply(get_winner, axis=1)

# Loop through every match
for index, match in matches.iterrows():
    # Update Current Ratings
    home_team_current_rating = current_ratings[match.home_team]
    away_team_current_rating = current_ratings[match.away_team]
    
    # Get New Ratings
    home_team_new_rating = calculate_rating_change(home_team_current_rating, 
                                                   away_team_current_rating, 
                                                   get_result_value(match.home_team, match.winner))
    
    away_team_new_rating = calculate_rating_change(away_team_current_rating, 
                                                   home_team_current_rating, 
                                                   get_result_value(match.away_team, match.winner))
    
    # Update Current Rating Dict
    current_ratings[match.home_team] = home_team_new_rating
    current_ratings[match.away_team] = away_team_new_rating
        
    # Update DF
    matches.at[index, 'home_team_current_rating'] = home_team_current_rating
    matches.at[index, 'away_team_current_rating'] = away_team_current_rating
    matches.at[index, 'home_team_new_rating'] = home_team_new_rating
    matches.at[index, 'away_team_new_rating'] = away_team_new_rating

matches.tail(5)

,yearly_match_number,round,date,time,venue,home_team,home_final_score,away_team,away_final_score,home_team_current_rating,away_team_current_rating,home_team_new_rating,away_team_new_rating,winner
15735,175,Semi Final,2020-10-09,18:50:00,Carrara,Richmond,80.0,St Kilda,49.0,1702,1492,1705,1488,Richmond
15736,176,Semi Final,2020-10-10,18:40:00,Gabba,Geelong,100.0,Collingwood,32.0,1659,1609,1666,1602,Geelong
15737,177,Preliminary Final,2020-10-16,19:20:00,Adelaide Oval,Port Adelaide,40.0,Richmond,46.0,1626,1705,1620,1711,Richmond
15738,178,Preliminary Final,2020-10-17,18:40:00,Gabba,Brisbane Lions,42.0,Geelong,82.0,1571,1666,1566,1672,Geelong
15739,179,Grand Final,2020-10-24,18:30:00,Gabba,Richmond,81.0,Geelong,50.0,1711,1672,1719,1665,Richmond


Output data to file

In [19]:
matches.to_pickle('../data/aflmatches_elo.pypickle')